In [1]:
from drone_data.ml_utils.general_functions import select_columns
from drone_data.ml_utils.mls_functions import CVRegression
import pandas as pd
from itertools import compress
import numpy as np

In [2]:
def select_dates(df, datestoselect):
    colsbol = [i.endswith(datestoselect[0]
    ) for i in df.columns]
    if len(datestoselect) > 0:
        for cols in range(1, len(datestoselect)):
            colsbol = np.array(colsbol) | np.array([i.endswith(
                datestoselect[cols]) for i in df.columns])

    return df[list(compress(df.columns,  colsbol))]


def save_prediction_results(model, kfolds, verbose = True):
    
    model.cv_fit(kfolds, verbose =verbose)
    y_pred = model.cv_prediction()
    tablemetrics = pd.concat(model.cv_metrics_score(y_pred))

    tablemetrics["features"] = '-'.join(model.variables_names)
    tablemetrics["model"] = model.model_name
    tablemetrics['n_features'] =  len(model.variables_names)
    return tablemetrics

### Config

In [3]:
verbose = False
testidsplant = ['327', '586', '270', '178',  '903',  '1110', '976', '1015', '137', '347', '433', '1029', '1072', '96', '636', '984', '423',
 '51', '514', '889', '803', '1027', '20_last', '843', '4', '794', '986', '1063', '619', '990', '5_first', '970', '156', '3',
 '479', '931', '953', '299', '904', '1064', '337', '830', '168', '833', '42', '5', '60', '678', '589', '712', '1114', '321',
 '160', '644', '801', '368', '932', '557', '487', '913', '25', '605', '883', '513', '322', '305', '10', '621', '928', '188',
 '648', '123', '11', '385', '728', '549', '674', '1019', '183', '508', '807', '1115', '383']



targetvar = 'total_w_x'
FILENAME = 'data/tanashi_cultivar_weigths_tp_phenomics_211220_vcenteredimgred_70_ph90_filt_.csv'
TIMEWINDOW = 'data/timpepoints_intervals.csv'
availablemodels = ['pls','svr_linear','rf', 'ridge','lasso']

OUTPUT_FN_SINGLE = 'single_time_regression'
OUTPUT_FN_MULTI = 'multi_time_regression'
OUTPUT_FILTER_REGRESSION = 'filtering_regression'
OUTPUT_TIMEWINDOW = 'reduced_time_points'

REDUCTION = FILENAME[(FILENAME.index('imgred_')+len('imgred_')):FILENAME.index('_ph9')]
REDUCTION

BESTFEATURES = ["blue_ms_q_0.5","nir_q_0.5",
             "grvi_rgb_q_0.5",
             "red_ms_q_0.5","rgbvi_rgb_q_0.5","savi_q_0.5","volume"]

dict_typevars = {'first_level': ['blue_eq_q_0.5','red_eq_q_0.5','green_eq_q_0.5',
                                 'blue_ms_q_0.5','red_ms_q_0.5','green_ms_q_0.5','nir_q_0.5','edge_q_0.5'],
                'second_level': ['rgbvi_q_0.5',
                                 'rgbvi_rgb_q_0.5',
                                 
                                 'grvi_rgb_q_0.5',
                                 'savi_q_0.5','ndvi_q_0.5','ndre_q_0.5','gndvi_q_0.5'],
                '3d': ['z_q_0.9','leaf_angle_q_0.5','volume']}
                
FEATURESNAMES = { 'first_level':
        {
                'all_features': dict_typevars['first_level'],
                'best': ["blue_ms_q_0.5","red_ms_q_0.5","nir_q_0.5"]
        },
        'second_level':
        {
                'all_features': dict_typevars['second_level'],
                'best': ["savi_q_0.5","grvi_rgb_q_0.5","rgbvi_rgb_q_0.5"]
        },
        '3d':
        {
                'all_features': dict_typevars['3d'],
                'best': ["volume"]
        }
}


df = pd.read_csv(FILENAME)

t = 25
testindexes = [df.loc[df.id_plant == i].index[0] for i in df.id_plant.values if i in testidsplant]



### Feature Filtering Results



#### Regression per level

In [4]:
from drone_data.ml_utils.mls_functions import CVRegression, set_model
import os


verbose = False


## Number 4 was removed because thehere is a problem with imagery
for model_name in availablemodels:
    modelsresults = []
    for feat_level in list(FEATURESNAMES.keys()):
        for featurefilter in list(FEATURESNAMES[feat_level].keys()):
            df_colsoi = select_columns(df,
            FEATURESNAMES[feat_level][featurefilter])
            print(FEATURESNAMES[feat_level][featurefilter])
            inputdata = select_dates(df_colsoi, datestoselect= ['t{}'.format(t)])
            ydata = df[[targetvar,'id_plant']].copy()

            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids]
            yinput = ydata.iloc[notnaids][targetvar]
            yinput = yinput*10   

            mlmodel = set_model(model_name,cv= 5)
            
            modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                                val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                                testids_fixed = testindexes)

            table_metrics = save_prediction_results(modelcv, kfolds = 5, verbose=verbose)
            table_metrics['t'] = t
            table_metrics['level'] = feat_level
            table_metrics['type_filter'] = featurefilter
            table_metrics['model'] = modelcv.model_name
            
            if verbose:
                print("time_point: {} R squared: {:.3f}".format(t,table_metrics['r2'].mean()))

            modelsresults.append(table_metrics)
            print('*'*10, 'checkpoint')

    pd.concat(modelsresults).reset_index().to_csv(os.path.join('regression_results/feature_selection','{}_{}.csv'.format(
        OUTPUT_FILTER_REGRESSION, modelcv.model_name)))


['blue_eq_q_0.5', 'red_eq_q_0.5', 'green_eq_q_0.5', 'blue_ms_q_0.5', 'red_ms_q_0.5', 'green_ms_q_0.5', 'nir_q_0.5', 'edge_q_0.5']
839
['blue_ms_q_0.5', 'red_ms_q_0.5', 'nir_q_0.5']
839
['rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5']
839
['savi_q_0.5', 'grvi_rgb_q_0.5', 'rgbvi_rgb_q_0.5']
839
['z_q_0.9', 'leaf_angle_q_0.5', 'volume']
839
['volume']
839
['blue_eq_q_0.5', 'red_eq_q_0.5', 'green_eq_q_0.5', 'blue_ms_q_0.5', 'red_ms_q_0.5', 'green_ms_q_0.5', 'nir_q_0.5', 'edge_q_0.5']
839
['blue_ms_q_0.5', 'red_ms_q_0.5', 'nir_q_0.5']
839
['rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5']
839
['savi_q_0.5', 'grvi_rgb_q_0.5', 'rgbvi_rgb_q_0.5']
839
['z_q_0.9', 'leaf_angle_q_0.5', 'volume']
839
['volume']
839
['blue_eq_q_0.5', 'red_eq_q_0.5', 'green_eq_q_0.5', 'blue_ms_q_0.5', 'red_ms_q_0.5', 'green_ms_q_0.5', 'nir_q_0.5', 'edge_q_0.5']
839
['blue_ms_q_0.5', 'red_ms_q_0.5', 

#### All variables vs best

In [13]:
allvariables_ = list(np.array([dict_typevars[i] for i in dict_typevars.keys()]).flatten())
allvariables = allvariables_[0]
for i in range(1, len(allvariables_)):
    allvariables+=allvariables_[i]
print(allvariables)
availablemodels = ['pls','svr_linear','rf', 'ridge','lasso']

## Number 4 was removed because thehere is a problem with imagery
for model_name in availablemodels:
    modelsresults = []
    for typefilter,features in zip(['all','best'],[allvariables,BESTFEATURES]):
        
            df_colsoi = select_columns(df,features)
            print(features)
            inputdata = select_dates(df_colsoi, datestoselect= ['t{}'.format(t)])
            ydata = df[[targetvar,'id_plant']].copy()

            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids]
            yinput = ydata.iloc[notnaids][targetvar]
            yinput = yinput*10   

            mlmodel = set_model(model_name,cv= 5)
            
            modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                                val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                                testids_fixed = testindexes)

            table_metrics = save_prediction_results(modelcv, kfolds = 5, verbose=verbose)
            table_metrics['t'] = t
            table_metrics['level'] = 'all'
            table_metrics['type_filter'] = typefilter
            table_metrics['model'] = modelcv.model_name
            
            if verbose:
                print("time_point: {} R squared: {:.3f}".format(t,table_metrics['r2'].mean()))

            modelsresults.append(table_metrics)
            print('*'*10, 'checkpoint')

    pd.concat(modelsresults).reset_index().to_csv(os.path.join('regression_results/feature_selection','{}_{}_allvsbest.csv'.format(
        OUTPUT_FILTER_REGRESSION, modelcv.model_name)))


C:\Users\aguilar\AppData\Local\Temp/ipykernel_23924/2141876460.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  allvariables_ = list(np.array([dict_typevars[i] for i in dict_typevars.keys()]).flatten())


['blue_eq_q_0.5', 'red_eq_q_0.5', 'green_eq_q_0.5', 'blue_ms_q_0.5', 'red_ms_q_0.5', 'green_ms_q_0.5', 'nir_q_0.5', 'edge_q_0.5', 'rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5', 'z_q_0.9', 'leaf_angle_q_0.5', 'volume', 'rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5', 'z_q_0.9', 'leaf_angle_q_0.5', 'volume']
['blue_eq_q_0.5', 'red_eq_q_0.5', 'green_eq_q_0.5', 'blue_ms_q_0.5', 'red_ms_q_0.5', 'green_ms_q_0.5', 'nir_q_0.5', 'edge_q_0.5', 'rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5', 'z_q_0.9', 'leaf_angle_q_0.5', 'volume', 'rgbvi_q_0.5', 'rgbvi_rgb_q_0.5', 'grvi_rgb_q_0.5', 'savi_q_0.5', 'ndvi_q_0.5', 'ndre_q_0.5', 'gndvi_q_0.5', 'z_q_0.9', 'leaf_angle_q_0.5', 'volume']
839
********** checkpoint
['blue_ms_q_0.5', 'nir_q_0.5', 'grvi_rgb_q_0.5', 'red_ms_q_0.5', 'rgbvi_rgb_q_0.5', 'savi_q_0.5', 'volume']
839
**

### Multi - temporal prediction

#### Single-time regression

In [ ]:

availablemodels = ['pls','svr_linear','rf', 'ridge','lasso']
## Number 4 was removed because thehere is a problem with imagery
for model_name in availablemodels:
    df_colsoi = select_columns(df,BESTFEATURES)
    
    modelsresults = []
    for t in list(range(0,26)):
        
        inputdata = select_dates(df_colsoi, datestoselect= ['t{}'.format(t)])
        ydata = df[[targetvar,'id_plant']].copy()

        notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
        xinput = inputdata.iloc[notnaids]
        yinput = ydata.iloc[notnaids][targetvar]
        yinput = yinput*10    
        mlmodel = set_model(model_name,cv= 5)
        modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                               val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                               testids_fixed = testindexes)

        table_metrics = save_prediction_results(modelcv, kfolds = 5)
        table_metrics['t'] = t
        verbose = True
        if verbose:
            print("time_point: {} R squared: {:.3f}".format(t,table_metrics['r2'].mean()))

        modelsresults.append(table_metrics)
        print('*'*10, 'checkpoint')
    pd.concat(modelsresults).reset_index().to_csv(os.path.join('regression_results','{}_{}.csv'.format(
        OUTPUT_FN_SINGLE, modelcv.model_name)))


#### Multi-temporal regression

In [14]:
availablemodels = ['pls','svr_linear','rf', 'ridge','lasso']
## Number 4 was removed because thehere is a problem with imagery
for model_name in availablemodels:
    print('*'*10, model_name)
    df_colsoi = select_columns(df,BESTFEATURES)
    
    modelsresults = []
    for t in list(range(0,26)) :

        datestobeselec = ['t{}'.format(ts) for ts in range(0,(t+1))]
        inputdata = select_dates(df_colsoi, datestoselect= datestobeselec)
        print(datestobeselec)
        print(inputdata.columns)
        print(len(inputdata.columns))
        ydata = df[[targetvar,'id_plant']].copy()

        notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
        xinput = inputdata.iloc[notnaids]
        yinput = ydata.iloc[notnaids][targetvar]
        yinput = yinput*10
        mlmodel = set_model(model_name,cv= 5)
        modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                                    val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                                    testids_fixed = testindexes)

        table_metrics = save_prediction_results(modelcv, kfolds = 5, verbose=verbose)
        table_metrics['t'] = t

        if verbose:
            print("time_point: {} R squared: {:.3f}".format(t,table_metrics['r2'].mean()))

        modelsresults.append(table_metrics)
        print('*'*10, 'checkpoint')
    pd.concat(modelsresults).reset_index().to_csv(os.path.join('regression_results','{}_{}.csv'.format(
        OUTPUT_FN_MULTI, modelcv.model_name)))

********** pls
['t0']
Index(['blue_ms_q_0.5_t0', 'grvi_rgb_q_0.5_t0', 'nir_q_0.5_t0',
       'red_ms_q_0.5_t0', 'rgbvi_rgb_q_0.5_t0', 'savi_q_0.5_t0', 'volume_t0'],
      dtype='object')
7
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
134 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
134 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\cross_decomposition\_pls.py", line 353, in fit
    pinv2(np.dot(self.x_loadings_.T, self.x_weights_), check_finite=False),
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-package

********** checkpoint
['t0', 't1']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'grvi_rgb_q_0.5_t0',
       'grvi_rgb_q_0.5_t1', 'nir_q_0.5_t0', 'nir_q_0.5_t1', 'red_ms_q_0.5_t0',
       'red_ms_q_0.5_t1', 'rgbvi_rgb_q_0.5_t0', 'rgbvi_rgb_q_0.5_t1',
       'savi_q_0.5_t0', 'savi_q_0.5_t1', 'volume_t0', 'volume_t1'],
      dtype='object')
14
839
********** checkpoint
['t0', 't1', 't2']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'blue_ms_q_0.5_t2',
       'grvi_rgb_q_0.5_t0', 'grvi_rgb_q_0.5_t1', 'grvi_rgb_q_0.5_t2',
       'nir_q_0.5_t0', 'nir_q_0.5_t1', 'nir_q_0.5_t2', 'red_ms_q_0.5_t0',
       'red_ms_q_0.5_t1', 'red_ms_q_0.5_t2', 'rgbvi_rgb_q_0.5_t0',
       'rgbvi_rgb_q_0.5_t1', 'rgbvi_rgb_q_0.5_t2', 'savi_q_0.5_t0',
       'savi_q_0.5_t1', 'savi_q_0.5_t2', 'volume_t0', 'volume_t1',
       'volume_t2'],
      dtype='object')
21
839
********** checkpoint
['t0', 't1', 't2', 't3']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'blue_ms_q_0.5_t2',
       'blue_ms_q_0.5_t3', 'grvi_

c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.132e+04, tolerance: 4.992e+04
  model = cd_fast.enet_coordinate_descent(


********** checkpoint
['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'blue_ms_q_0.5_t10',
       'blue_ms_q_0.5_t11', 'blue_ms_q_0.5_t12', 'blue_ms_q_0.5_t13',
       'blue_ms_q_0.5_t14', 'blue_ms_q_0.5_t15', 'blue_ms_q_0.5_t16',
       'blue_ms_q_0.5_t17',
       ...
       'volume_t21', 'volume_t22', 'volume_t23', 'volume_t3', 'volume_t4',
       'volume_t5', 'volume_t6', 'volume_t7', 'volume_t8', 'volume_t9'],
      dtype='object', length=168)
168
839
********** checkpoint
['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'blue_ms_q_0.5_t10',
       'blue_ms_q_0.5_t11', 'blue_ms_q_0.5_t12', 'blue_ms_q_0.5_t13',
       'blue_ms_q_0.5_t14', 'blue_ms_q_0.5_t15', 'blue_m

c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.521e+04, tolerance: 5.065e+04
  model = cd_fast.enet_coordinate_descent(


********** checkpoint
['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25']
Index(['blue_ms_q_0.5_t0', 'blue_ms_q_0.5_t1', 'blue_ms_q_0.5_t10',
       'blue_ms_q_0.5_t11', 'blue_ms_q_0.5_t12', 'blue_ms_q_0.5_t13',
       'blue_ms_q_0.5_t14', 'blue_ms_q_0.5_t15', 'blue_ms_q_0.5_t16',
       'blue_ms_q_0.5_t17',
       ...
       'volume_t23', 'volume_t24', 'volume_t25', 'volume_t3', 'volume_t4',
       'volume_t5', 'volume_t6', 'volume_t7', 'volume_t8', 'volume_t9'],
      dtype='object', length=182)
182
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.160e+04, tolerance: 5.065e+04
  model = cd_fast.enet_coordinate_descent(
c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.051e+04, tolerance: 4.891e+04
  model = cd_fast.enet_coordinate_descent(


********** checkpoint


### Time-interval prediction

In [13]:
from drone_data.ml_utils.general_functions import SplitIds
from drone_data.ml_utils.mls_functions import check_arraytype, get_eval_metrics,set_model

import os

### readinf timewindow info
dfsepw = pd.read_csv(TIMEWINDOW)

uniquewindows = np.unique(dfsepw.windowsize)
uniquewindows = list(uniquewindows)
### reading predicting data
df = pd.read_csv(FILENAME)
df_colsoi = select_columns(df,BESTFEATURES)
uniquewindows

['15 days\r\n(7)',
 '30 days\r\n(4)',
 '7 days\r\n(14)',
 'All\r\ndates\r\n(26)',
 'Growth\r\nStages\r\n(5)']

#### CV model

In [21]:

for model_name in availablemodels:
    
    allresults = []
    df_colsoi = select_columns(df,BESTFEATURES)
    
    print('*'*10, model_name)
    for z in range(len(uniquewindows)):
        
        timeslist = list(dfsepw[dfsepw.windowsize == uniquewindows[z]].codcolumns.values)
        print(timeslist)
        for t in range(1,len(timeslist)+1):
            datestoselect = list(np.array(timeslist)[list(range(0,t))])
            inputdata = select_dates(df_colsoi, datestoselect= datestoselect)
            print(len(inputdata.columns))
            ydata = df[[targetvar,'id_plant']].copy()
            
            ### removing nas
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids].to_numpy()
            yinput = ydata.iloc[notnaids][targetvar].values
            
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids]
            yinput = ydata.iloc[notnaids][targetvar]
            yinput = yinput*10
           

            mlmodel = set_model(model_name,cv= 5)
            modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                                        val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                                        testids_fixed = testindexes)

            table_metrics = save_prediction_results(modelcv, kfolds = 5, verbose=verbose)

            table_metrics['windowtype'] = uniquewindows[z]
            table_metrics['time_points'] = t
            table_metrics['seq_time_points'] = '-'.join(datestoselect)
            table_metrics["features"] = '-'.join(inputdata.columns)
            table_metrics["model"] = model_name
            table_metrics['n_features'] =  len(inputdata.columns)
            print('R squared: ', np.round(np.mean(table_metrics['r2']),2))
            allresults.append(table_metrics)
            
        pd.concat(allresults).reset_index().to_csv(os.path.join('regression_results/sequenced_time_points','{}_{}.csv'.format(
        OUTPUT_TIMEWINDOW, modelcv.model_name)))


********** pls
['t0', 't3', 't8', 't13', 't19', 't23', 't25']
7
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
134 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
134 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\cross_decomposition\_pls.py", line 353, in fit
    pinv2(np.dot(self.x_loadings_.T, self.x_weights_), check_finite=False),
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-package

R squared:  0.09
14
839
R squared:  0.38
21
839
R squared:  0.48
28
839
R squared:  0.61
35
839
R squared:  0.68
42
839
R squared:  0.71
49
839
R squared:  0.72
['t0', 't8', 't19', 't25']
7
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
134 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
134 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\cross_decomposition\_pls.py", line 353, in fit
    pinv2(np.dot(self.x_loadings_.T, self.x_weights_), check_finite=False),
  File "c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-package

R squared:  0.09
14
839
R squared:  0.47
21
839
R squared:  0.66
28
839
R squared:  0.68
['t0', 't1', 't3', 't5', 't7', 't10', 't12', 't15', 't18', 't20', 't22', 't23', 't24', 't25']
7
839


...

R squared:  0.72
168
839
R squared:  0.72
175
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.521e+04, tolerance: 5.065e+04
  model = cd_fast.enet_coordinate_descent(


R squared:  0.72
182
839


c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.160e+04, tolerance: 5.065e+04
  model = cd_fast.enet_coordinate_descent(
c:\Users\aguilar\miniconda3\envs\drone_data\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.051e+04, tolerance: 4.891e+04
  model = cd_fast.enet_coordinate_descent(


R squared:  0.72
['t0', 't5', 't13', 't22', 't25']
7
839
R squared:  0.09
14
839
R squared:  0.41
21
839
R squared:  0.59
28
839
R squared:  0.65
35
839
R squared:  0.67


In [15]:
for model_name in availablemodels:
    allresults = []
    df_colsoi = select_columns(df,BESTFEATURES)
    
    allresults = []
    print('*'*10, model_name)
    for z in range(len(uniquewindows)):
        
        timeslist = list(dfsepw[dfsepw.windowsize == uniquewindows[z]].codcolumns.values)
        print(timeslist)
        for t in range(1,len(timeslist)+1):
            datestoselect = list(np.array(timeslist)[list(range(0,t))])
            inputdata = select_dates(df_colsoi, datestoselect= datestoselect)
            print(len(inputdata.columns))
            ydata = df[[targetvar,'id_plant']].copy()
            
            ### removing nas
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids].to_numpy()
            yinput = ydata.iloc[notnaids][targetvar].values
            
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids]
            yinput = ydata.iloc[notnaids][targetvar]
            yinput = yinput*10

********** pls
['t0', 't3', 't9', 't14', 't20', 't24', 't26']
7
14
21
28
35
42
49
['t0', 't9', 't20', 't26']
7
14
21
28
['t0', 't1', 't2', 't6', 't8', 't11', 't13', 't16', 't19', 't21', 't23', 't24', 't25', 't26']
7
14
21
28
35
42
49
56
63
70
77
84
91
98
['t0', 't1', 't2', 't3', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26']
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
168
175
182
['t0', 't6', 't14', 't23', 't26']
7
14
21
28
35
********** svr_linear
['t0', 't3', 't9', 't14', 't20', 't24', 't26']
7
14
21
28
35
42
49
['t0', 't9', 't20', 't26']
7
14
21
28
['t0', 't1', 't2', 't6', 't8', 't11', 't13', 't16', 't19', 't21', 't23', 't24', 't25', 't26']
7
14
21
28
35
42
49
56
63
70
77
84
91
98
['t0', 't1', 't2', 't3', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26'

In [17]:
mlmodel = set_model(model_name,cv= 5)
            
mlmodel = set_model(model_name,cv= 5)
modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                            val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                            testids_fixed = testindexes)
tr_x, tr_y, test_x, test_y = modelcv.get_xyvaldata(modelcv.X, 
                                                               modelcv.Y, phase= 'validation')

In [4]:
availablemodels

['pls', 'svr_linear', 'rf', 'ridge', 'lasso']

#### Testing prediction

In [14]:

allresults = []
for model_name in ['rf', 'ridge', 'lasso']:
    allresults = []
    df_colsoi = select_columns(df,BESTFEATURES)
    
    allresults = []
    print('*'*10, model_name)
    for z in range(len(uniquewindows)):
        
        timeslist = list(dfsepw[dfsepw.windowsize == uniquewindows[z]].codcolumns.values)
        print(timeslist)
        for t in range(1,len(timeslist)+1):
            datestoselect = list(np.array(timeslist)[list(range(0,t))])
            inputdata = select_dates(df_colsoi, datestoselect= datestoselect)
            print(len(inputdata.columns))
            ydata = df[[targetvar,'id_plant']].copy()
            
            ### removing nas
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids].to_numpy()
            yinput = ydata.iloc[notnaids][targetvar].values
            
            notnaids = np.where(np.logical_not(np.isnan(ydata[targetvar])))
            xinput = inputdata.iloc[notnaids]
            yinput = ydata.iloc[notnaids][targetvar]
            yinput = yinput*10
           
            mlmodel = set_model(model_name,cv= 5)
            
            mlmodel = set_model(model_name,cv= 5)
            modelcv = CVRegression(xinput, yinput, mlmodel, model_name = model_name,
                                        val_perc =None, test_perc = 10,seed = 123, shuffle = True, 
                                        testids_fixed = testindexes)
            


            ### get data
            tr_x, tr_y, test_x, test_y = modelcv.get_xyvaldata(modelcv.X, 
                                                               modelcv.Y, phase= 'validation')
            tr_x = check_arraytype(tr_x, typeinput = 'input')
            tr_y = check_arraytype(tr_y)
            ## fit model
            mlmodel.fit(tr_x, tr_y)
            ## model evaluation
            test_x = check_arraytype(test_x, typeinput = 'input')
            ypred = mlmodel.predict(test_x)
            test_y = check_arraytype(test_y)
            pdmetric = get_eval_metrics(test_y, ypred)

            pdmetric['windowtype'] = uniquewindows[z]
            pdmetric['seq_time_points'] = '-'.join(datestoselect)
            pdmetric["features"] = '-'.join(inputdata.columns)
            pdmetric["model"] = model_name
            pdmetric['n_features'] =  len(inputdata.columns)

            allresults.append(pdmetric)
    
    pd.concat(allresults).reset_index().to_csv(os.path.join('regression_results/sequenced_time_points','{}_testing_dataset_{}.csv'.format(
        OUTPUT_TIMEWINDOW, modelcv.model_name)))


********** rf
['t0', 't3', 't8', 't13', 't19', 't23', 't25']
7
839
14
839
21
839
28
839
35
839
42
839
49
839
['t0', 't8', 't19', 't25']
7
839
14
839
21
839
28
839
['t0', 't1', 't3', 't5', 't7', 't10', 't12', 't15', 't18', 't20', 't22', 't23', 't24', 't25']
7
839
14
839
21
839
28
839
35
839
42
839
49
839
56
839
63
839
70
839
77
839
84
839
91
839
98
839
['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25']
7
839
14
839
21
839
28
839
35
839
42
839
49
839
56
839
63
839
70
839
77
839
84
839
91
839
98
839
105
839
112
839
119
839
126
839
133
839
140
839
147
839
154
839
161
839
168
839
175
839
182
839
['t0', 't5', 't13', 't22', 't25']
7
839
14
839
21
839
28
839
35
839
********** ridge
['t0', 't3', 't8', 't13', 't19', 't23', 't25']
7
839
14
839
21
839
28
839
35
839
42
839
49
839
['t0', 't8', 't19', 't25']
7
839
14
839
21
839
28
839
['t0', 't1', 't3', 't5', 't7', 't10', 't12', '